In [1]:
import pandas as pd, numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ChromeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from datetime import timedelta
import pathlib
import math
import time

Compute coms of newly aquired articles from srf

In [2]:
# get the date of the moste recent article in the merged dataset
liverss_and_coms_path = pathlib.Path.home() / 'Desktop' / 'SocialMediaMonitor' / 'data' / 'raw' / 'liverss_and_coms_srf.csv'
liverss_and_coms_df = pd.read_csv(liverss_and_coms_path)
# print(liverss_and_coms_df)
# print(liverss_and_coms_df.columns)
liverss_and_coms_df['published'] = pd.to_datetime(liverss_and_coms_df['published'], errors = 'coerce')
most_recent_article = liverss_and_coms_df['published'].max()
print(most_recent_article)

# get the ids of the articles that are less than 15 days old
recent_date = pd.to_datetime(most_recent_article-timedelta(15), utc=True)
mask = (liverss_and_coms_df['published'] > recent_date)  # & (total_df['is_com_or_sub'] == 'sub') -> not yet implemented
recent_articles = liverss_and_coms_df.loc[mask]
recent_articles_id = recent_articles['id'].tolist()
print(recent_articles_id[:3])
print(len(recent_articles_id))

2022-05-08 18:35:00+00:00
['https://www.srf.ch/sport/fussball/super-league/4-0-heimsieg-gegen-servette-luzern-mindestens-in-der-barrage-und-nur-noch-3-punkte-hinter-gc', 'https://www.srf.ch/sport/fussball/super-league/3-0-heimsieg-gegen-lausanne-luzern-feiert-wichtigen-erfolg-im-abstiegskampf', 'https://www.srf.ch/news/abstimmungen-15-mai-2022/abstimmung-stadt-luzern-braucht-luzern-mehr-zusammenhaengende-velorouten']
6


In [3]:
# Configure the Chrome webdriver
driver = webdriver.Chrome('/users/michaellappert/Downloads/chromedriver') # Needs to be adjusted client independently

# get the commenents of those articles
new_coms_df = pd.DataFrame(columns = ['author', 'published', 'summary', 'id', 'is_com_or_sub', 'link'])

for Id, link in zip(recent_articles['id'], recent_articles['link']):
    print(link)
    driver.get(link)
    time.sleep(1)
    
    while True:
        try:
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME, "button.show-more-bar__child.show-more-bar__child--grow.js-show-more-button.button--white.button--mixed-case.button--link-text-color.button--no-spacing.button--no-text-on-mobile"))).click()
            print("LOAD MORE RESULTS button clicked")
        except TimeoutException:
            try:
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME, "comment__link.comment__link--expand"))).click()
            except TimeoutException:  
                print("No more LOAD MORE RESULTS button to be clicked")
                break
            
    # get authors
    authors = driver.find_elements_by_class_name("comment__user")
    authors_list = [author.get_attribute('innerHTML') for author in authors]
    authors_list = [item.replace('Kommentar von','')\
                    .replace('\n','')\
                    .replace('<span class="h-offscreen"> ','')\
                    .replace('<span>', '')\
                    .replace('</span>', '')\
                    .replace('&nbsp;', '')\
                    .replace('  ', '') for item in authors_list]
    #authors_list = [item for item in authors_list if item != '']
    # print('authors:', authors_list[:5])


    # get publication dates
    published_date = driver.find_elements_by_class_name("comment__time")
    date_list = [pd.to_datetime(date.get_attribute('data-timestamp'), unit='s', utc=True) for date in published_date]
    # print('date_list: ', date_list[:5])
    
    # get text bodys
    bodys = driver.find_elements_by_class_name("comment__content")
    body_list = [body.get_attribute('innerHTML') for body in bodys]
    body_list = [item.replace('<br>', '').replace('\n','') for item in body_list]
    for item in body_list:
        if item == 'Beliebteste Kommentare werden geladen.':
            body_list.remove(item)
    # print('body_list:\n', body_list[:5])
    # print(body_list[:5])

    # assert to make sure that we have all the data for every com
    # may fail if the webdriver didn't wait for the page to load
    assert len(body_list) == len(date_list) == len(authors_list), f"Assertion problem, probably due to page parsing before page is fully loaded : bodys:{body_list} dates:{date_list} authors:{authors_list}"
    
    temp_df = pd.DataFrame(list(zip(authors_list, date_list, body_list)), columns =['author', 'published', 'summary'])
    temp_df['id'] = Id
    temp_df['is_com_or_sub'] = 'com'
    temp_df['link'] = link
    temp_df.drop_duplicates(ignore_index=True, inplace=True)
    print(f'{len(temp_df)} comments found')
    if len(temp_df) > 0:
        new_coms_df = pd.concat([new_coms_df, temp_df], ignore_index=True, axis=0)

new_coms_df.head()
driver.quit()
    

https://www.srf.ch/sport/fussball/super-league/4-0-heimsieg-gegen-servette-luzern-mindestens-in-der-barrage-und-nur-noch-3-punkte-hinter-gc
LOAD MORE RESULTS button clicked
No more LOAD MORE RESULTS button to be clicked
9 comments found
https://www.srf.ch/sport/fussball/super-league/3-0-heimsieg-gegen-lausanne-luzern-feiert-wichtigen-erfolg-im-abstiegskampf
No more LOAD MORE RESULTS button to be clicked
0 comments found
https://www.srf.ch/news/abstimmungen-15-mai-2022/abstimmung-stadt-luzern-braucht-luzern-mehr-zusammenhaengende-velorouten
No more LOAD MORE RESULTS button to be clicked
0 comments found
https://www.srf.ch/sport/mehr-sport/handball/mitten-in-den-playoffs-perkovac-bei-kriens-luzern-per-sofort-freigestellt
No more LOAD MORE RESULTS button to be clicked
0 comments found
https://www.srf.ch/sport/fussball/super-league/basel-schlaegt-luzern-mit-3-0-spielverderber-stocker-fcz-meisterparty-muss-noch-warten
LOAD MORE RESULTS button clicked
No more LOAD MORE RESULTS button to be c

In [9]:
# filtering for newest coms and getting newest new_last_check_coms
# srf_article_coms_df = new_coms_df[new_coms_df['date'] > last_coms_check] -> not yet implemented
srf_article_coms_df = new_coms_df
srf_article_coms_df.head()

,author,published,summary,id,is_com_or_sub,link
0,Josef Graf(Josef Graf),2022-05-09 09:55:57+00:00,Zum x-ten Mal machten die Joker der Luzerner d...,https://www.srf.ch/sport/fussball/super-league...,com,https://www.srf.ch/sport/fussball/super-league...
1,Stefan Gisler(GiJaBra),2022-05-08 21:22:17+00:00,"Luzern überrascht nun doch noch, wäre cool wen...",https://www.srf.ch/sport/fussball/super-league...,com,https://www.srf.ch/sport/fussball/super-league...
2,Ueli Hugetober (Ueli.67),2022-05-09 07:02:54+00:00,"Nein, wäre es nicht!",https://www.srf.ch/sport/fussball/super-league...,com,https://www.srf.ch/sport/fussball/super-league...
3,Francis Waeber(Francis Waeber),2022-05-08 17:34:46+00:00,"Wau, das war nicht zu erwarten. Wie schonmal g...",https://www.srf.ch/sport/fussball/super-league...,com,https://www.srf.ch/sport/fussball/super-league...
4,Marco Meier (ma.meier),2022-05-08 19:52:40+00:00,Die Spitze war langweilig in den letzten Jahre...,https://www.srf.ch/sport/fussball/super-league...,com,https://www.srf.ch/sport/fussball/super-league...


In [7]:
# Saving generated data to raw data folder
srf_article_coms_df.to_csv(r'~/Desktop/SocialMediaMonitor/data/raw/liverss_comments_srf.csv', index = False)